<a href="https://colab.research.google.com/github/toby-p/w266-final-project/blob/main/Saving_models_to_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Saving models to drive

Adapted from [this notebook](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb), along with [this tutorial on saving to Google Drive](https://medium.com/@ml_kid/how-to-save-our-model-to-google-drive-and-reuse-it-2c1028058cb2).


## Setup

### Installs and imports

Install and import TensorFlow and dependencies:

In [ ]:
# !pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import datetime
import os

from google.colab import drive
import tensorflow as tf
from tensorflow import keras

# Mount Google Drive:
drive.mount("/content/gdrive")

# Directory where models will be stored in GDrive:
MODEL_DIR = "/content/gdrive/MyDrive/models"

# Make the directories for storing results if they don't exist yet:
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)


def gdrive_save_dir(*subdir: str, model_name: str = "test_model"): 
    """Create timestamped directory in GDrive for storing checkpoints or models.
    
    Args:
        subdir: optional subdirectories of the main model directory
            (e.g. `checkpoints`, `final_model`, etc.)
        model_name: main name for directory specifying the model being saved.
    """
    model_dir = f"{MODEL_DIR}/{model_name}"
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    for s in subdir:
        model_dir = f"{model_dir}/{s}"
        if not os.path.exists(model_dir):
            os.mkdir(model_dir)
    now = datetime.datetime.now()
    now_str = now.strftime("%Y_%m_%d__%H_%M_%S")
    dir_path = f"{model_dir}/{now_str}"
    os.mkdir(dir_path)
    print(f"Created checkpoint dir: {dir_path}")
    return dir_path


gdrive_save_dir("checkpoints", model_name = "test_model")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Created checkpoint dir: /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_39


'/content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_39'

### Get an example dataset

To demonstrate how to save and load weights, you'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). To speed up these runs, use the first 1000 examples:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Define a model

Start by building a simple sequential model:

In [ ]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

You can use a trained model without having to retrain it, or pick-up training where you left off in case the training process was interrupted. The `tf.keras.callbacks.ModelCheckpoint` callback allows you to continually save the model both *during* and at *the end* of training.

### Checkpoint callback usage

Create a `tf.keras.callbacks.ModelCheckpoint` callback that saves weights only during training:

In [ ]:
checkpoint_dir = gdrive_save_dir("checkpoints", model_name = "test_model")
checkpoint_path = f"{checkpoint_dir}/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Created checkpoint dir: /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_41
Epoch 1/10
20/32 [=================>............] - ETA: 0s - loss: 1.3779 - sparse_categorical_accuracy: 0.6203 
Epoch 1: saving model to /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_41/cp.ckpt
32/32 [==============================] - 1s 11ms/step - loss: 1.1311 - sparse_categorical_accuracy: 0.6810 - val_loss: 0.7109 - val_sparse_categorical_accuracy: 0.7660
Epoch 2/10
22/32 [===================>..........] - ETA: 0s - loss: 0.4469 - sparse_categorical_accuracy: 0.8793
Epoch 2: saving model to /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_41/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4206 - sparse_categorical_accuracy: 0.8840 - val_loss: 0.5175 - val_sparse_categorical_accuracy: 0.8330
Epoch 3/10
23/32 [====================>.........] - ETA: 0s - loss: 0.2983 - sparse_categorical_accuracy: 0.9198
Epoch 3: s

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [ ]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'cp.ckpt.data-00000-of-00001', 'checkpoint']

As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights. 

Now rebuild a fresh, untrained model and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [ ]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.4060 - sparse_categorical_accuracy: 0.0940 - 204ms/epoch - 6ms/step
Untrained model, accuracy:  9.40%


Then load the weights from the checkpoint and re-evaluate:

In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4308 - sparse_categorical_accuracy: 0.8660 - 119ms/epoch - 4ms/step
Restored model, accuracy: 86.60%


### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_dir = gdrive_save_dir("checkpoints", model_name = "test_model")
checkpoint_path = checkpoint_dir + "/cp-{epoch:04d}.ckpt"

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)

Created checkpoint dir: /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_44

Epoch 5: saving model to /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_44/cp-0005.ckpt

Epoch 10: saving model to /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_44/cp-0010.ckpt

Epoch 15: saving model to /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_44/cp-0015.ckpt

Epoch 20: saving model to /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_44/cp-0020.ckpt

Epoch 25: saving model to /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_44/cp-0025.ckpt

Epoch 30: saving model to /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_44/cp-0030.ckpt

Epoch 35: saving model to /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_44/cp-0035.ckpt

Epoch 40: saving model to /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_2

Now, review the resulting checkpoints and choose the latest one:

In [ ]:
os.listdir(checkpoint_dir)

['cp-0000.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'checkpoint']

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'/content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_44/cp-0050.ckpt'

Note: The default TensorFlow format only saves the 5 most recent checkpoints.

To test, reset the model, and load the latest checkpoint:

In [ ]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4774 - sparse_categorical_accuracy: 0.8760 - 163ms/epoch - 5ms/step
Restored model, accuracy: 87.60%


## What are these files?

The above code stores the weights to a collection of [checkpoint](../../guide/checkpoint.ipynb)-formatted files that contain only the trained weights in a binary format. Checkpoints contain:
* One or more shards that contain your model's weights.
* An index file that indicates which weights are stored in which shard.

If you are training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

## Manually save weights

To save weights manually, use `tf.keras.Model.save_weights`. By default, `tf.keras`—and the `Model.save_weights` method in particular—uses the TensorFlow [Checkpoint](../../guide/checkpoint.ipynb) format with a `.ckpt` extension. To save in the HDF5 format with a `.h5` extension, refer to the [Save and load models](https://www.tensorflow.org/guide/keras/save_and_serialize) guide.

In [ ]:
checkpoint_dir = gdrive_save_dir("checkpoints", model_name = "test_model")

# Save the weights
model.save_weights(f'{checkpoint_dir}/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights(f'{checkpoint_dir}/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

Created checkpoint dir: /content/gdrive/MyDrive/models/test_model/checkpoints/2022_07_17__20_20_53
32/32 - 0s - loss: 0.4774 - sparse_categorical_accuracy: 0.8760 - 154ms/epoch - 5ms/step
Restored model, accuracy: 87.60%


## Save the entire model

Call `tf.keras.Model.save` to save a model's architecture, weights, and training configuration in a single `file/folder`. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

An entire model can be saved in two different file formats (`SavedModel` and `HDF5`). The TensorFlow `SavedModel` format is the default file format in TF2.x. However, models can be saved in `HDF5` format. More details on saving entire models in the two file formats is described below.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model), [HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([Saved Model](https://www.tensorflow.org/lite/models/convert/#convert_a_savedmodel_recommended_), [HDF5](https://www.tensorflow.org/lite/models/convert/#convert_a_keras_model_))

\*Custom objects (for example, subclassed models or layers) require special attention when saving and loading. Refer to the **Saving custom objects** section below.

### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using `tf.keras.models.load_model` and are compatible with TensorFlow Serving. The [SavedModel guide](../../guide/saved_model.ipynb) goes into detail about how to `serve/inspect` the SavedModel. The section below illustrates the steps to save and restore the model.

In [ ]:
model_dir = gdrive_save_dir("full_model", model_name = "test_model")

# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
model.save(model_dir) 

Created checkpoint dir: /content/gdrive/MyDrive/models/test_model/full_model/2022_07_17__20_20_54
Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1708 - sparse_categorical_accuracy: 0.6590
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4204 - sparse_categorical_accuracy: 0.8840
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2913 - sparse_categorical_accuracy: 0.9330
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2050 - sparse_categorical_accuracy: 0.9510
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1632 - sparse_categorical_accuracy: 0.9640
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/models/test_model/full_model/2022_07_17__20_20_54/assets


The SavedModel format is a directory containing a protobuf binary and a TensorFlow checkpoint. Inspect the saved model directory:

In [ ]:
os.listdir(model_dir)

['variables', 'assets', 'saved_model.pb', 'keras_metadata.pb']

Reload a fresh Keras model from the saved model:

In [ ]:
new_model = tf.keras.models.load_model(model_dir)

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4392 - sparse_categorical_accuracy: 0.8600 - 160ms/epoch - 5ms/step
Restored model, accuracy: 86.00%
(1000, 10)
